In [ ]:
# ==============================================================================
# IMPORTAÇÃO DE BIBLIOTECAS
# ==============================================================================
print("Iniciando o projeto de Detecção de Fraudes...")
print("Carregando bibliotecas...")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Para a Etapa 2: Preparação
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Para a Etapa 4 e 5: Modelagem e Avaliação
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    precision_recall_curve,
    average_precision_score
)

# Configurações de visualização
sns.set_theme(style="whitegrid", palette="pastel")
print("Bibliotecas carregadas com sucesso.\n")

# ==============================================================================
# ETAPA 1: Entendimento do Problema e dos Dados
# ==============================================================================
print("--- INICIANDO ETAPA 1: Entendimento do Problema e dos Dados ---")

# Carregar o conjunto de dados a partir do arquivo .xlsb
file_name = "mercadoFinanceiro.xlsb"
try:
    df = pd.read_excel(file_name, engine='pyxlsb', sheet_name=0)
    print(f"Arquivo '{file_name}' carregado com sucesso!\n")
except FileNotFoundError:
    print(f"Erro: O arquivo '{file_name}' não foi encontrado.")
    exit()
except ImportError:
    print("Erro: A biblioteca 'pyxlsb' é necessária. Instale-a com: pip install pyxlsb")
    exit()

# Exploração Inicial dos Dados
print(df.head())

print("\n2. Informações Gerais do DataFrame (tipos de dados e valores nulos):")
df.info()

print("\n3. Resumo Estatístico das Variáveis Numéricas:")
print(df.describe().T)

# VERIFICAÇÃO CRÍTICA: Identificar a coluna Alvo e o Desbalanceamento
target_column = None
possible_targets = ['Class', 'fraude', 'isFraud', 'Fraud', 'is_fraud', 'FRAUDE']
for col in possible_targets:
    if col in df.columns:
        target_column = col
        break

if not target_column:
    print("\nERRO CRÍTICO: Não foi possível identificar a coluna alvo (ex: 'Class', 'fraude').")
    exit()

# --- CORREÇÃO APLICADA (PARTE 1) ---
# Converter a coluna alvo para tipo numérico (inteiro).
# Isso corrige o erro 'ValueError' e garante que os modelos de ML
# recebam 0 e 1 como números, não como texto.
try:
    df[target_column] = df[target_column].astype(int)
    print(f"\nColuna alvo '{target_column}' encontrada e convertida para tipo numérico (inteiro).")
except ValueError as e:
    print(f"\nERRO ao converter a coluna alvo para inteiro: {e}")
    print("Verifique os dados na coluna alvo.")
    exit()
# --- FIM DA CORREÇÃO (PARTE 1) ---

print(f"\n4. Análise de Desbalanceamento da Coluna Alvo ('{target_column}'):")
class_counts = df[target_column].value_counts()
class_perc = df[target_column].value_counts(normalize=True) * 100

print(f"Contagem de Classes:\n{class_counts}")
print(f"\nPercentual de Classes:\n{class_perc}")

# Visualização do Desbalanceamento
plt.figure(figsize=(8, 6))

# --- CORREÇÃO APLICADA (PARTE 2) ---
# Atualizada a sintaxe do Seaborn conforme o aviso (deprecation warning)
# Agora que a coluna é numérica, a paleta {0:..., 1:...} funciona.
ax = sns.countplot(
    x=target_column,
    data=df,
    hue=target_column,  # Adicionado conforme aviso
    palette={0: "#a0d", 1: "#f0a"},  # Paleta com chaves numéricas
    legend=False  # Adicionado conforme aviso
)
# --- FIM DA CORREÇÃO (PARTE 2) ---

plt.title('Distribuição das Classes (0 = Normal, 1 = Fraude)', fontsize=16)
plt.xlabel('Classe')
plt.ylabel('Contagem')
# Adicionar anotações de percentual
total = len(df)
for p in ax.patches:
    percentage = '{:.4f}%'.format(100 * p.get_height() / total)
    x = p.get_x() + p.get_width() / 2
    y = p.get_height()
    ax.annotate(percentage, (x, y), ha='center', va='bottom', fontsize=12)
plt.show()

print("\n--- FIM DA ETAPA 1 ---")

# ==============================================================================
# ETAPA 2: Preparação dos Dados
# ==============================================================================
print("\n--- INICIANDO ETAPA 2: Preparação dos Dados ---")

# 1. Tratamento de Valores Faltantes
print(f"1. Verificando valores nulos: {df.isnull().sum().any()}")

# 2. Normalização e Padronização
if 'Time' in df.columns:
    df = df.drop('Time', axis=1)
    print("2. Coluna 'Time' removida.")

# Separar features (X) e alvo (y)
X = df.drop(target_column, axis=1)
y = df[target_column]  # 'y' agora é garantidamente numérico

# Padronizar as features (X)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

print("3. Todas as features foram padronizadas (StandardScaler).")
print("Visualização das features padronizadas:")
print(X_scaled.head())

print("\n--- FIM DA ETAPA 2 ---")

# ==============================================================================
# ETAPA 3: Análise Exploratória de Dados (EDA)
# ==============================================================================
print("\n--- INICIANDO ETAPA 3: Análise Exploratória de Dados ---")

# 1. Análise de Correlação
print("1. Analisando correlação das features com a Fraude...")
corr_matrix = df.corr()
corr_target = corr_matrix[target_column].sort_values(ascending=False)
print("Correlações mais fortes com a Fraude:")
print(corr_target.head(5))
print(corr_target.tail(5))

# 2. Visualização das Features Principais
if 'Amount' in df.columns:
    print("\n2. Visualizando distribuição de 'Amount' por classe...")
    plt.figure(figsize=(12, 6))

    # --- CORREÇÃO APLICADA (PARTE 3) ---
    # Aplicando a mesma correção de sintaxe do Seaborn ao boxplot
    sns.boxplot(
        x=target_column,
        y='Amount',
        data=df,
        showfliers=False,
        hue=target_column,  # Adicionado
        palette={0: "#a0d", 1: "#f0a"},  # Chaves numéricas
        legend=False  # Adicionado
    )
    # --- FIM DA CORREÇÃO (PARTE 3) ---

    plt.title('Distribuição do Valor da Transação (Amount) por Classe', fontsize=16)
    plt.xlabel('Classe (0 = Normal, 1 = Fraude)')
    plt.ylabel('Valor da Transação (Amount)')
    plt.show()
else:
    print("\n2. Coluna 'Amount' não encontrada para visualização.")

print("\n--- FIM DA ETAPA 3 ---")

# ==============================================================================
# ETAPA 4: Seleção de Modelos e Algoritmos
# ==============================================================================
print("\n--- INICIANDO ETAPA 4: Seleção de Modelos e Algoritmos ---")

# 1. Divisão de Dados em Treinamento e Teste
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # Essencial para dados desbalanceados
)

print("1. Dados divididos em Treino (80%) e Teste (20%) com estratificação.")
print(f"   - Fraudes no treino: {sum(y_train)} ({sum(y_train) / len(y_train) * 100:.2f}%)")
print(f"   - Fraudes no teste: {sum(y_test)} ({sum(y_test) / len(y_test) * 100:.2f}%)")

# 2. Escolha de Algoritmos
models = {
    "Regressão Logística": LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000),
    "Random Forest": RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1)
}
print("\n2. Modelos selecionados: Regressão Logística e Random Forest (com 'class_weight=balanced').")

print("\n--- FIM DA ETAPA 4 ---")

# ==============================================================================
# ETAPA 5: Treinamento e Avaliação do Modelo
# ==============================================================================
print("\n--- INICIANDO ETAPA 5: Treinamento e Avaliação do Modelo ---")

results = {}

# Loop para treinar e avaliar cada modelo
for name, model in models.items():
    print(f"\n--- Treinando e Avaliando: {name} ---")

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    # Avaliação
    print("Métricas de Avaliação (Foco em Recall e Precisão):")
    print(classification_report(y_test, y_pred, target_names=['Normal (0)', 'Fraude (1)']))

    auc_roc = roc_auc_score(y_test, y_proba)
    auc_pr = average_precision_score(y_test, y_proba)  # Mais importante em dados desbalanceados

    results[name] = {
        "AUC-ROC": auc_roc,
        "AUC-PR (Média Precisão)": auc_pr
    }

    print(f"AUC-ROC (distingue classes): {auc_roc:.4f}")
    print(f"AUC-PR (foco na classe positiva): {auc_pr:.4f}")

    # Visualização de Resultados (Etapa 7)

    # 1. Matriz de Confusão
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Prev. Normal', 'Prev. Fraude'],
                yticklabels=['Real Normal', 'Real Fraude'])
    plt.title(f'Matriz de Confusão - {name}', fontsize=16)
    plt.ylabel('Verdadeiro')
    plt.xlabel('Previsto')
    plt.show()

    print(f"Análise da Matriz de Confusão ({name}):")
    print(f"   - Verdadeiros Negativos (Normais detectados): {cm[0][0]}")
    print(f"   - Falsos Positivos (Normais bloqueados): {cm[0][1]} <- Ruim para experiência")
    print(f"   - Falsos Negativos (Fraudes perdidas): {cm[1][0]} <- Ruim para o banco")
    print(f"   - Verdadeiros Positivos (Fraudes detectadas): {cm[1][1]}")

# 2. Curva Precision-Recall (Etapa 7)
plt.figure(figsize=(10, 7))
for name, model in models.items():
    y_proba = model.predict_proba(X_test)[:, 1]
    precision, recall, _ = precision_recall_curve(y_test, y_proba)
    avg_prec = average_precision_score(y_test, y_proba)
    plt.plot(recall, precision, label=f'{name} (AUC-PR = {avg_prec:.3f})')

plt.title('Curva Precision-Recall (P-R)', fontsize=16)
plt.xlabel('Recall (Sensibilidade - % de fraudes pegas)')
plt.ylabel('Precision (% de alertas corretos)')
plt.legend()
plt.show()

print("\n--- Comparação Final do Desempenho dos Modelos ---")
results_df = pd.DataFrame(results).T
print(results_df.sort_values(by="AUC-PR (Média Precisão)", ascending=False))

print("\n--- FIM DA ETAPA 5 ---")

# ==============================================================================
# ETAPA 6 e 7: Ajuste Final e Interpretação
# ==============================================================================
print("\n--- INICIANDO ETAPA 6 e 7: Ajuste Final e Interpretação ---")

# 1. Refinamento (Ajuste Final)
best_model = models["Random Forest"]
print("1. Modelo 'Random Forest' selecionado como final para interpretação.")

# 2. Interpretação dos Resultados (Feature Importance)
importances = best_model.feature_importances_
feature_names = X.columns
feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values('importance', ascending=False)

print("\n2. Importância das Features (O que o modelo olha?):")
print(feature_importance_df.head(10))

# 3. Criação de Visualização Impactante (Etapa 7)
plt.figure(figsize=(12, 8))
sns.barplot(
    x='importance',
    y='feature',
    data=feature_importance_df.head(10),
    palette='viridis'
)
plt.title('Top 10 Features Mais Importantes para Detecção de Fraude', fontsize=16)
plt.xlabel('Nível de Importância')
plt.ylabel('Feature')
plt.show()

# 4. Relatório Final e Comunicação (Etapa 7)
print("\n--- Relatório Final e Recomendações (Tradução para Stakeholder) ---")
print("""
Insight 1: O Problema Central está Confirmado (Etapa 1)
Os dados são extremamente desbalanceados. Com 99.82% de transações normais contra 0.17% de
fraudes:

A métrica Acurácia é inútil. Um modelo que sempre diz "Normal" teria 99.82% de acurácia,
mas seria um fracasso.

O sucesso do projeto depende 100% das métricas Precisão e Recall.

Insight 2: (Etapa 3)
A análise de correlação mostra que nenhuma variável sozinha é capaz de prever fraude.
As correlações mais fortes (ex: V4 com 0.036 e V12 com -0.040) são muito fracas.

Conclusão: É impossível criar uma regra simples (ex: "se V12 < -5, é fraude").
Isso prova a necessidade de um modelo de Machine Learning complexo, como o Random Forest,
que é capaz de encontrar padrões combinando múltiplas variáveis.

Insight 3: A Batalha dos Modelos - O Trade-Off Crítico (Etapa 5)
Este é o insight mais importante para o gestor de risco. Você tem dois modelos com duas
estratégias totalmente diferentes:

A) Regressão Logística (O "Guarda Agressivo")

Recall (Fraude): 87% - EXCELENTE. Ele pegou 85 das 98 fraudes. O banco perdeu pouco
dinheiro (Falsos Negativos: 13).

Precisão (Fraude): 2% - PÉSSIMO. Para pegar essas 85 fraudes, ele bloqueou 5.239 clientes
inocentes (Falsos Positivos).

Conclusão de Negócio: Este modelo é inviável. Ele cumpre a meta de "pegar fraudes", mas
 destrói a experiência do cliente, indo contra a segunda parte do briefing.

B) Random Forest (O "Cirurgião Preciso")

Recall (Fraude): 62% - BOM. Ele pegou 61 das 98 fraudes. Não é tão bom quanto o outro modelo,
pois 37 fraudes passaram (Falsos Negativos).

Precisão (Fraude): 86% - EXCEPCIONAL. O modelo tem altíssima confiança. Quando ele diz "isto é
fraude", ele está certo 86% das vezes. Ele bloqueou apenas 10 clientes inocentes.

Conclusão de Negócio: Este modelo é excelente e viável. Ele equilibra perfeitamente as
duas metas do briefing: protege o cliente (poucos falsos positivos) e ainda captura a
maioria das fraudes.

Insight 4: A Métrica Correta Confirma a Escolha (Etapa 5)
A "Comparação Final" mata a charada.

O AUC-ROC é parecido para os dois (0.94), pois ele é "enganado" pela enorme quantidade de
casos normais (classe 0).

A AUC-PR (Média Precisão) é a métrica mais importante em dados desbalanceados.

Random Forest (0.77) é astronomicamente melhor que a Regressão Logística (0.20) nesta métrica,
confirmando que é o modelo superior para este problema.

Insight 5: Agora Sabemos Onde Olhar (Etapa 7)
O Random Forest nos diz quais são as pistas. As 5 features mais importantes que o modelo usa
para "decidir" se algo é fraude são:

V12

V4

V3

V14

V11

Mesmo que essas features sejam anonimizadas (comuns em dados financeiros por privacidade),
a equipe de risco agora sabe que as combinações dessas 5 variáveis são os indicadores mais
fortes de fraude.

Resumo da Conclusão (Para o Gestor de Risco):
"Conseguimos construir um modelo (Random Forest) que atinge o equilíbrio perfeito entre
segurança e experiência do cliente. Ele bloqueia apenas 10 clientes inocentes para cada
56.000 transações e, ao mesmo tempo, identifica corretamente 86% dos alertas que gera,
capturando mais de 60% de todas as tentativas de fraude. O modelo aprendeu que as variáveis V12,
 V4 e V3 são os principais indicadores de risco."
""")

print("\n--- PROJETO CONCLUÍDO ---")